In [1]:
import sys
sys.path.append('../utils/')

import libraries
from libraries import *
import utils_functions
from utils_functions import *

%matplotlib inline

reload(libraries)
from libraries import *
reload(utils_functions)
from utils_functions import *

# checking
print(get_available_gpus())
print(psutil.virtual_memory())
print(keras.__version__)

Using TensorFlow backend.


[]
svmem(total=17179869184, available=6309834752, percent=63.3, used=16002539520, free=33968128, active=6469238784, inactive=6275866624, wired=3257434112)
2.0.8


In [52]:
target_size = (256, 256) #(512, 512)

output = '../output'
data = 'compressed_data'
cd_path = join(join(output, data), str(target_size[0]) + '_' + str(target_size[1]))

# loading test data

x_test = load_array(join(cd_path, 'x_test'))
y_test = load_array(join(cd_path, 'y_test'))

# models to include

models_list = ['resnet_50', 'inception_v3', 'xception', 'mobilenet', 
               'vgg_16', 'vgg_19',
               'densenet_121', 'densenet_161', 'densenet_169']
models_list = ['resnet_50']

preds = np.empty((0,))
for i in models_list:
    print(join(output, i))
    model_ouput = join(output, i)
    pred = load_array(join(join(output, i), 'predictions'))
    preds = np.append(preds, pred)

print(preds.shape)
# preds_mean = preds.mean(axis=0)
# preds_mean

../output/resnet_50
(160,)


In [71]:
_FLOATX = 'float32'
_EPSILON = 10e-8
_IMAGE_DATA_FORMAT = 'channels_last'


def epsilon():
    """Returns the value of the fuzz factor used in numeric expressions.
    # Returns
        A float.
    # Example
    """
    return _EPSILON

def _to_tensor(x, dtype):
    """Convert the input `x` to a tensor of type `dtype`.
    # Arguments
        x: An object to be converted (numpy array, list, tensors).
        dtype: The destination type.
    # Returns
        A tensor.
    """
    x = tf.convert_to_tensor(x)
    if x.dtype != dtype:
        x = tf.cast(x, dtype)
    return x

def clip(x, min_value, max_value):
    """Element-wise value clipping.
    # Arguments
        x: Tensor or variable.
        min_value: Python float or integer.
        max_value: Python float or integer.
    # Returns
        A tensor.
    """
    if max_value is not None and max_value < min_value:
        max_value = min_value
    if max_value is None:
        max_value = np.inf
    min_value = _to_tensor(min_value, x.dtype.base)
    max_value = _to_tensor(max_value, x.dtype.base)
    return tf.clip_by_value(x, min_value, max_value)

def binary_crossentropy(target, output, from_logits=False):
    """Binary crossentropy between an output tensor and a target tensor.
    # Arguments
        target: A tensor with the same shape as `output`.
        output: A tensor.
        from_logits: Whether `output` is expected to be a logits tensor.
            By default, we consider that `output`
            encodes a probability distribution.
    # Returns
        A tensor.
    """
    _epsilon = _EPSILON = 10e-8
    # Note: tf.nn.sigmoid_cross_entropy_with_logits
    # expects logits, Keras expects probabilities.
    if not from_logits:
        # transform back to logits
#         _epsilon = _to_tensor(epsilon(), output.dtype.base) # HERE: .base_dtype for .base
        output = tf.clip_by_value(output, _epsilon, 1 - _epsilon)
        output = tf.log(output / (1 - output))

    return tf.nn.sigmoid_cross_entropy_with_logits(labels=target,
                                                   logits=output)

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(clip(y_true * y_pred, 0, 1))) # HERE K.clip for clip
    possible_positives = K.sum(K.round(clip(y_true, 0, 1)))      # HERE K.clip for clip
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(clip(y_true * y_pred, 0, 1)))   # HERE K.clip for clip
    predicted_positives = K.sum(K.round(clip(y_pred, 0, 1)))       # HERE K.clip for clip
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def fbeta_score(y_true, y_pred, beta=1):
    """Computes the F score.

    The F score is the weighted harmonic mean of precision and recall.
    Here it is only computed as a batch-wise average, not globally.

    This is useful for multi-label classification, where input samples can be
    classified as sets of labels. By only using accuracy (precision) a model
    would achieve a perfect score by simply assigning every class to every
    input. In order to avoid this, a metric should penalize incorrect class
    assignments as well (recall). The F-beta score (ranged from 0.0 to 1.0)
    computes this, as a weighted mean of the proportion of correct class
    assignments vs. the proportion of incorrect class assignments.

    With beta = 1, this is equivalent to a F-measure. With beta < 1, assigning
    correct classes becomes more important, and with beta > 1 the metric is
    instead weighted towards penalizing incorrect class assignments.
    """
    if beta < 0:
        raise ValueError('The lowest choosable beta is zero (only precision).')

    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(clip(y_true, 0, 1))) == 0:       # HERE K.clip for clip
        return 0

    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    bb = beta ** 2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

def fmeasure(y_true, y_pred):
    """Computes the f-measure, the harmonic mean of precision and recall.

    Here it is only computed as a batch-wise average, not globally.
    """
    return fbeta_score(y_true, y_pred, beta=1)

def kullback_leibler_divergence(y_true, y_pred):
    """Computes the KLdivergence between prediction and target values.
    """
    y_true = clip(y_true, K.epsilon(), 1)           # HERE K.clip for clip
    y_pred = clip(y_pred, K.epsilon(), 1)           # HERE K.clip for clip
    return K.mean(K.sum(y_true * K.log(y_true / y_pred), axis=-1))

def matthews_correlation(y_true, y_pred):
    """Matthews correlation metric.

    It is only computed as a batch-wise average, not globally.

    Computes the Matthews correlation coefficient measure for quality
    of binary classification problems.
    """
    y_pred_pos = K.round(clip(y_pred, 0, 1))     # HERE K.clip for clip
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(clip(y_true, 0, 1))            # HERE K.clip for clip
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

In [72]:
metric_func = [binary_crossentropy, binary_accuracy, recall, precision, 
               fbeta_score, fmeasure, hinge, squared_hinge, kullback_leibler_divergence,
               poisson, cosine_proximity, matthews_correlation]

metrics = {}
for i in metric_func:
    print(str(i.__name__))
    metrics[i.__name__] = i(preds, y_test)

print("Metrics for the validation set")

for i, j in metrics.items():
    print(i, j)

p = precision(preds, y_test)
p.eval

binary_crossentropy
binary_accuracy
recall
precision
fbeta_score
fmeasure
hinge
squared_hinge
kullback_leibler_divergence
poisson
cosine_proximity
matthews_correlation
Metrics for the validation set
binary_crossentropy Tensor("logistic_loss_29:0", shape=(160,), dtype=float64)
binary_accuracy Tensor("Mean_74:0", shape=(), dtype=float32)
recall Tensor("truediv_148:0", shape=(), dtype=float64)
precision Tensor("truediv_149:0", shape=(), dtype=float64)
fbeta_score Tensor("truediv_152:0", shape=(), dtype=float64)
fmeasure Tensor("truediv_155:0", shape=(), dtype=float64)
hinge Tensor("Mean_75:0", shape=(), dtype=float64)
squared_hinge Tensor("Mean_76:0", shape=(), dtype=float64)
kullback_leibler_divergence Tensor("Mean_77:0", shape=(), dtype=float64)
poisson Tensor("Mean_78:0", shape=(), dtype=float64)
cosine_proximity Tensor("Neg_9:0", shape=(), dtype=float64)
matthews_correlation Tensor("truediv_157:0", shape=(), dtype=float64)


<bound method Tensor.eval of <tf.Tensor 'truediv_158:0' shape=() dtype=float64>>

In [86]:
sess = tf.Session()
loss = metrics['binary_crossentropy']
np.mean(loss.eval(session=sess))

sess = tf.Session()
loss = metrics['binary_accuracy']
np.mean(loss.eval(session=sess))


# print(preds)
# print(y_test)

0.0